In [19]:
import numpy as np

## 1.1

In [20]:
def prod_by_p(P, M):
    return M[P]


def pivot(A):
    n = A.shape[1]
    P = [[float(i == j) for i in range(n)] for j in range(n)]
    change_steps = 0
    for i in range(n):
        row = max(range(i, n), key=lambda j: abs(A[j, i]))
        if i != row:
            P[i], P[row] = P[row], P[i]
            change_steps += 1
        P = np.argwhere(np.array(P) == 1)[:, 1]
        return np.array(P), change_steps


def get_lu(A):
    n = A.shape[1]

    P, num_swaps = pivot(A)
    A = prod_by_p(P, A)

    L = np.eye(n)
    U = np.zeros((n, n))

    for j in range(n):
        for i in range(j+1):
            s1 = sum(U[k, j] * L[i, k] for k in range(i))
            U[i, j] = A[i, j] - s1
        for i in range(j, n):
            s2 = sum(U[k, j] * L[i, k] for k in range(j))
            L[i, j] = (A[i, j]-s2) / U[j, j]
    return (P, L, U), num_swaps


def get_lu_solution(L, U, b):
    n = b.shape[0]
    z = np.zeros(n)
    x = np.zeros(n)

    z[0] = b[0]
    for i in range(1, n):
        z[i] = b[i] - sum([L[i, j]*z[j] for j in range(i)])

    x[n-1] = z[n-1] / U[n-1, n-1]
    for i in reversed(range(n-1)):
        x[i] = (z[i] - sum([U[i, j]*x[j] for j in range(i+1, n)])) / U[i, i]

    return x


def det(U, s):
    return (-1)**s * np.prod([U[i, i] for i in range(U.shape[0])])


def inv_matrix(L, U, P):
    n = L.shape[0]
    P = prod_by_p(P, np.eye(n))
    A_inv = np.zeros((n, n))
    for idx, row in enumerate(P):
        A_inv[idx] = get_lu_solution(L, U, row)
    return A_inv.T


A = np.array([
    [-5, -1, -3, -1],
    [-2,  0,  8, -4],
    [-7, -2,  2, -2],
    [2,  -4, -4,  4]
])

b = np.array([18, -12, 6, -12]).astype(np.float32)

(p, l, u), swaps = get_lu(A)

print(f"P:\n{p}")
print(f"L:\n{l}")
print(f"U:\n{u}")

x = get_lu_solution(l, u, prod_by_p(p, b))
print(f"x:\n{x}")

print(f"B:\n{b}\nMy B:\n{np.dot(A, x)}")

det_u = det(u, swaps)
print(f"det(A): {det_u}")

a_inv = inv_matrix(l, u, p)
print(f"A^(-1):\n{a_inv}")
print(f"A * A^(-1):\n{np.dot(A, a_inv)}")


P:
[2 1 0 3]
L:
[[ 1.          0.          0.          0.        ]
 [ 0.28571429  1.          0.          0.        ]
 [ 0.71428571  0.75        1.          0.        ]
 [-0.28571429 -8.         -5.6         1.        ]]
U:
[[ -7.          -2.           2.          -2.        ]
 [  0.           0.57142857   7.42857143  -3.42857143]
 [  0.           0.         -10.           3.        ]
 [  0.           0.           0.          -7.2       ]]
x:
[-2.  3. -3. -2.]
B:
[ 18. -12.   6. -12.]
My B:
[ 18. -12.   6. -12.]
det(A): 288.0
A^(-1):
[[ 0.22222222  0.22222222 -0.33333333  0.11111111]
 [-0.33333333 -0.33333333  0.25       -0.29166667]
 [-0.33333333 -0.08333333  0.25       -0.04166667]
 [-0.77777778 -0.52777778  0.66666667 -0.13888889]]
A * A^(-1):
[[ 1.00000000e+00  3.33066907e-16 -7.77156117e-16  8.32667268e-17]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-1.77635684e-15 -1.77635684e-15  4.8849

## 1.2

In [24]:
def tridiagonal_matrix(ac, bc, cc, fc):
    n = len(f)
    ac, bc, cc, fc = map(np.array, (a, b, c, f))

    for it in range(1, n):
        mc = ac[it-1]/bc[it-1]
        bc[it] = bc[it] - mc*cc[it-1]
        fc[it] = fc[it] - mc*fc[it-1]

    xc = bc
    xc[-1] = fc[-1]/bc[-1]

    for il in range(n-2, -1, -1):
        xc[il] = (fc[il]-cc[il]*xc[il+1])/bc[il]

    return xc


c = [-9, -4, -8, 5]
b = [18, -9, 21, -10, 12]
a = [2, -9, -4, 7]
f = [-81, 71, -39, 64, 3]

print("answer:", tridiagonal_matrix(a, b, c, f))

A = np.array([
    [18, -9,  0,   0,  0],
    [2,  -9, -4,   0,  0],
    [0,  -9, 21,  -8,  0],
    [0,   0, -4, -10,  5],
    [0,   0,  0,   7, 12]
])
print(f"f:\n{f}\nmy f:\n{np.dot(A, answer)}")


answer: [-8 -7 -6 -3  2]
f:
[-81, 71, -39, 64, 3]
my f:
[-81  71 -39  64   3]


## 1.3

In [30]:
def norm(M):
    n = len(M.shape)
    if n == 1:
        return np.sqrt(M.dot(M))
    elif n == 2:
        return np.linalg.norm(M)  # Frobenius norm


def cond_num(M):
    M_inv = np.linalg.inv(M)
    return norm(M) * norm(M_inv)


def iterations_method(a, b, eps, zeidel):
    n = len(b)
    alpha = np.zeros((n, n))
    beta = np.zeros(n)

    for i in range(n):
        beta[i] = b[i] / a[i, i]
        for j in range(n):
            if i != j:
                alpha[i, j] = - a[i, j] / a[i, i]

    cond_coef = norm(alpha)/(1 - norm(alpha))
    x_pr = np.zeros(n)
    x_cur = beta

    num_it = 0
    while (norm(x_cur - x_pr) * cond_coef >= eps):

        if not zeidel:
            x_pr = x_cur
            x_cur = beta + np.dot(alpha, x_cur)

        else:

            x_pr = x_cur
            tmp1 = np.zeros(n)

            tmp1[0] = beta[0] + sum(alpha[0] * x_cur)
            for k in range(1, n):
                tmp1[k] = beta[k] + \
                    sum(alpha[k, :k] * tmp1[:k]) + \
                    sum(alpha[k, k:] * x_cur[k:])

            x_cur = tmp1

        num_it += 1

    return x_cur, num_it


A = np.array([
    [21, -6, -9, -4],
    [-6, 20, -4, 2],
    [-2, -7, -20, 3],
    [4,  9, 6, 24]
])

b = np.array([127, -144, 236, -5]).astype(np.float32)
print("сondition number:", cond_num(A))
print("norm:", norm(A))

print("iterations")
x, iterations = iterations_method(A, b, 10e-3, zeidel=False)
print(f"iterations: {iterations}\nx: {x}")
print("Ax:", np.dot(A, x))

print("Zeidel")
x, iterations = iterations_method(A, b, 10e-3, zeidel=True)
print(f"iterations: {iterations}\nx: {x}")
print("Ax:", np.dot(A, x))


сondition number: 4.553464977796038
norm: 46.9148164229596
iterations
iterations: 13
x: [ 0.99969256 -9.00010663 -8.0001808   5.00006692]
Ax: [ 126.99554312 -143.99943092  236.00517809   -5.00166828]
Zeidel
iterations: 7
x: [ 1.00003705 -8.99997205 -8.00001895  4.99998808]
Ax: [ 127.00082862 -143.99961145  236.00007342   -5.        ]


## 1.4

In [33]:
def max_elem_idx(M):
    i_max, j_max, m = 0, 0, 0
    for i in range(len(M)):
        for j in range(i+1, len(M)):
            if np.abs(M[i, j]) > m:
                m = np.abs(M[i, j])
                i_max, j_max = i, j
    return i_max, j_max


def criteria(A):
    return np.sqrt(sum([A[i, j]**2 for i in range(len(A)) for j in range(len(A)) if i > j]))


def orth(A, i, j):
    if A[i, i] == A[j, j]:
        phi = np.pi / 4
    else:
        phi = .5 * np.arctan(2*A[i, j] / (A[i, i] - A[j, j]))

    U = np.eye(A.shape[0])
    U[i, j] = -np.sin(phi)
    U[j, i] = np.sin(phi)
    U[i, i] = np.cos(phi)
    U[j, j] = np.cos(phi)
    return U


def jacobi(A, eps=0.01):
    num_it = 1
    U_final = np.eye(A.shape[0])
    print("default criterion: ", criteria(A))
    while criteria(A) > eps:
        i, j = max_elem_idx(A)
        U = orth(A, i, j)
        U_final = np.dot(U_final, U)
        A = np.linalg.multi_dot([U.T, A, U])
        print(f"it: {num_it}, norm: {criteria(A)}")
        num_it += 1

    return A, U_final


A = np.array([
    [8, -3, 9],
    [-3, 8, -2],
    [9, -2, -8],
])

J, U = jacobi(A)
eigenvalues = [J[i, i] for i in range(len(J))]
eigenvectores = [U[:, i] for i in range(len(U))]

print("after iterations:", J, sep='\n')
print("eigenvalues:", eigenvalues, sep='\n')
print("Values:", eigenvectores, sep='\n')

print("orotogonality:")
for i in range(len(eigenvectores)):
    for j in range(i+1, len(eigenvectores)):
        print(f"(x{i}, x{j}) = ", np.dot(
            eigenvectores[i], eigenvectores[j]))

print("Ax = lambda*x")
for e, v in zip(eigenvalues, eigenvectores):
    print(f"{np.dot(A, v)} = {(e * v)}")


default criterion:  9.695359714832659
it: 1, norm: 3.6055512754639896
it: 2, norm: 0.5955119569103899
it: 3, norm: 0.29951665483432294
it: 4, norm: 0.008567321831287715
after iterations:
[[ 1.41143308e+01 -8.56676126e-03 -2.96902931e-18]
 [-8.56676126e-03  5.94541747e+00  9.80043440e-05]
 [ 6.78448118e-16  9.80043440e-05 -1.20597483e+01]]
eigenvalues:
[14.114330836196043, 5.945417474082699, -12.05974831027874]
Values:
[array([ 0.78387401, -0.50264086,  0.3645459 ]), array([0.47034713, 0.86395803, 0.17986134]), array([-0.40535802,  0.03047448,  0.91364992])]
orotogonality:
(x0, x1) =  1.3877787807814457e-17
(x0, x2) =  0.0
(x1, x2) =  0.0
Ax = lambda*x
[11.0598277  -7.10184067  5.14378058] = [11.06385705 -7.09443935  5.14532141]
[2.78965505 5.14090015 1.06631735] = [2.79641004 5.13659116 1.06935078]
[  4.88856174  -0.36742993 -11.01837051] = [  4.88851565  -0.3675146  -11.01838813]


## 1.5

In [39]:
def get_v(A, i):
    V = np.zeros(len(A))
    V[i] = A[i, i] + np.sign(A[i, i]) * \
        np.sqrt(sum([A[j, i]**2 for j in range(i, len(A))]))
    V[i+1:] = A[i+1:, i]
    return V.reshape(-1, 1)


def get_qr(A):
    n = len(A)
    Q = np.eye(n)
    for i in range(n-1):
        v = get_v(A, i)
        H = np.eye(n) - 2/(np.dot(v.T, v)) * np.dot(v, v.T)
        A = np.dot(H, A)
        Q = np.dot(Q, H)

    return Q, A


def criteria(A):
    n = len(A)
    return np.sqrt(sum([A[i, j]**2 for j in range(n) for i in range(j+1, n)]))


def comlex_ans(A):
    n = len(A)
    answers = np.zeros((n-1, 2), dtype=complex)
    for i in range(n-1):
        a11 = A[i][i]
        a12 = A[i][i + 1]
        a21 = A[i + 1][i]
        a22 = A[i + 1][i + 1]
        answers[i] = np.roots((1, -a11 - a22, a11 * a22 - a12 * a21))
    return answers


def complex_criteria(A, eps):
    Q, R = get_qr(A)
    A_next = np.dot(R, Q)

    complRoots = comlex_ans(A)
    complRootsNext = comlex_ans(A_next)

    for idx, (a, b) in enumerate(zip(complRoots, complRootsNext)):
        if np.sum(np.abs(a - b)**2) < eps:
            if 0 not in b:
                return set([idx, idx+1]), b
    return 0, 0


def iterate_qr(A,  eps=0.001):
    num_it = 1
    indSet = set(range(len(A)))
    while criteria(A) > eps:

        compIndices, ev = complex_criteria(A, eps)
        if compIndices != 0:
            i = indSet.difference(set(compIndices)).pop()
            print("Num iterations: ", num_it)
            return sorted(np.append(A[i, i], ev), reverse=True)

        Q, R = get_qr(A)
        A = np.dot(R, Q)
        num_it += 1
        if num_it == 10:
            break
    print("Num iterations: ", num_it)
    return [A[i, i] for i in range(len(A))]


A = np.array([
    [-9, -9, -3],
    [-9, 0, 2],
    [-5, -1, -4],
])

A = np.array([
    [1, 3, 1],
    [1, 1, 4],
    [4, 3, 1],
])


Q, R = get_qr(A)
print("cr:", criteria(A))
print("Q:", Q, sep='\n')
print("R:", R, sep='\n')
print("QR:", np.dot(Q, R), sep='\n')
print(f"Q-1:\n{np.linalg.inv(Q)}\nQ.T:\n{Q.T}")

e_values = iterate_qr(A)
print("eigen values:", e_values, sep='\n')
print("eigen values from numpy:", np.linalg.eig(A)[0], sep='\n')


cr: 5.0990195135927845
Q:
[[-0.23570226  0.96582428  0.10783277]
 [-0.23570226  0.05083286 -0.97049496]
 [-0.94280904 -0.25416428  0.21566555]]
R:
[[-4.24264069e+00 -3.77123617e+00 -2.12132034e+00]
 [ 1.48094774e-17  2.18581284e+00  9.14991422e-01]
 [ 1.10030137e-16  1.49547129e-16 -3.55848152e+00]]
QR:
[[1. 3. 1.]
 [1. 1. 4.]
 [4. 3. 1.]]
Q-1:
[[-0.23570226 -0.23570226 -0.94280904]
 [ 0.96582428  0.05083286 -0.25416428]
 [ 0.10783277 -0.97049496  0.21566555]]
Q.T:
[[-0.23570226 -0.23570226 -0.94280904]
 [ 0.96582428  0.05083286 -0.25416428]
 [ 0.10783277 -0.97049496  0.21566555]]
Num iterations:  5
eigen values:
[(6.313174860729379+0j), (-1.6761220268750678+1.5529441689226442j), (-1.6761220268750678-1.5529441689226442j)]
eigen values from numpy:
[ 6.34280359+0.j         -1.6714018 +1.55214776j -1.6714018 -1.55214776j]


## 2.1

In [43]:
def f(x):
    return x*np.exp(x) + x**2 - 1


def df(x):
    return np.exp(x)*(1 + x) + 2*x


def phi(x, lmbd=0.1):
    return x - lmbd*f(x)


def dphi(x, lmbd=0.1):
    return 1 - lmbd*df(x)


def getQ(l, r):
    return max(abs(dphi(l)), abs(dphi(r)))


def simple_iterations(a, b, eps=10e-4):
    x_pr, x = a, (a + b)/2
    q = getQ(a, b)
    coef = q/(1 - q)

    num_it = 1
    print("q:", q)
    while coef*abs(x - x_pr) > eps:
        print(f"it: {num_it}, dx: {coef*abs(x - x_pr)}")
        x_pr, x = x, phi(x)
        num_it += 1
    print(f"final it: {num_it}, dx: {coef*abs(x - x_pr)}")
    return x


def newton_method(a, b, x0=0.6, eps=10e-4):
    x_pr, x = 0, x0

    num_it = 1
    while abs(x - x_pr) > eps:
        print(f"it: {num_it}, dx: {abs(x - x_pr)}")
        x_pr, x = x, x - f(x)/df(x)
        num_it += 1
    print(f"final it: {num_it}, dx: {abs(x - x_pr)}")
    return x


print("simple iterations")
root = simple_iterations(0, 1)
print(f"root = {root}")
print(f"checking: f(root) = {f(root)}")

print("Newton method")
root = newton_method(0, 1)
print(f"root = {root}")
print(f"checking: f(root) = {f(root)}")


simple iterations
q: 0.9
it: 1, dx: 4.500000000000001
it: 2, dx: 0.0669245718150578
it: 3, dx: 0.04383309222432547
it: 4, dx: 0.028873130772942686
it: 5, dx: 0.019089521905553634
it: 6, dx: 0.012651770063255909
it: 7, dx: 0.008398522530342458
it: 8, dx: 0.005581030447877612
it: 9, dx: 0.003711340165028277
it: 10, dx: 0.002469161742569726
it: 11, dx: 0.0016432469489274795
it: 12, dx: 0.0010938193158787082
final it: 13, dx: 0.00072819528311735
root = 0.47833362186501366
checking: f(root) = 0.0005387003240253652
Newton method
it: 1, dx: 0.6
it: 2, dx: 0.11014053864985934
it: 3, dx: 0.011566112889711821
final it: 4, dx: 0.0001209380902851831
root = 0.47817241037014363
checking: f(root) = 4.38638101396549e-08


## 2.2

In [48]:
def f1(x):
    return 2*x[0] - np.cos(x[1])


def f2(x):
    return 2*x[1] - np.exp(x[0])


def phi1(x):
    return np.cos(x[1]) / 2


def phi2(x):
    return np.exp(x[0]) / 2


def df(x, func="", arg=""):
    if func == "f1":
        if arg == "x1":
            return 2
        else:
            return np.sin(x[1])
    elif func == "f2":
        if arg == "x1":
            return -np.exp(x[0])
        else:
            return 2

    elif func == "phi1":
        if arg == "x1":
            return 0
        else:
            return -np.sin(x[1])/2
    elif func == "phi2":
        if arg == "x1":
            return np.exp(x[0]) / 2
        else:
            return 0


def J(x):
    return np.array([[df(x, "f1", "x1"), df(x, "f1", "x2")],
                     [df(x, "f2", "x1"), df(x, "f2", "x2")]])


def A1(x):
    return np.array([[f1(x), df(x, "f1", "x2")],
                     [f2(x), df(x, "f2", "x2")]])


def A2(x):
    return np.array([[df(x, "f1", "x1"), f1(x)],
                     [df(x, "f2", "x1"), f2(x)]])


def get_Q(x):
    m = np.array([[df(x, func="phi1", arg="x1"), df(x, func="phi1", arg="x2")],
                  [df(x, func="phi2", arg="x1"), df(x, func="phi2", arg="x2")], ])
    return max(np.sum(abs(m), axis=1))


def get_root(intervals, eps=0.001, iters=False):
    x_old, x = np.zeros(2), np.mean(intervals, axis=1)
    num_it = 0

    if iters:
        coef = get_Q(x) / (1 - get_Q(x))
    else:
        coef = 1

    while np.max(abs(x - x_old)) * coef > eps:
        num_it += 1
        if iters:
            x_old, x = x, np.array([phi1(x), phi2(x)])
        else:
            x_old, x = x, np.array([x[0] - np.linalg.det(A1(x)) / np.linalg.det(J(x)),
                                    x[1] - np.linalg.det(A2(x)) / np.linalg.det(J(x))])
    print("iterations:", num_it)
    return x


intervals = np.array([[0, .5],
                      [.5, 1]])

print("Newton method")
x = get_root(intervals, iters=False)
print("Newton answer:", x)
print(f"f1(x) = {f1(x)}, f2(x) = {f2(x)}")

print("simple iterations")
x = get_root(intervals, iters=True)
print("simple iterations answer:", x)
print(f"f1(x) = {f1(x)}, f2(x) = {f2(x)}")

Newton method
iterations: 3
Newton answer: [0.37372831 0.72657115]
f1(x) = 5.555556015224283e-13, f2(x) = -5.792255564074367e-12
simple iterations
iterations: 9
simple iterations answer: [0.37370172 0.72629289]
f1(x) = -0.00023799730317619616, f2(x) = -0.0005178677490138561
